In [1]:
import cv2
from PIL import Image
import numpy as np

In [2]:
def get_limits(color):
    c = np.uint8([[color]])  # BGR values
    # uint8 — это тип данных «unsigned integer 8-bit» (беззнаковое целое 0–255). Мы говорим: «Пусть все числа будут целыми от 0 до 255»
    # [[color]] Так OpenCV «понимает» любой массив размером (height, width, channels) 
             # как изображение с высотой, шириной и каналами. Здесь у нас «картинка» всего 1×1 пиксель с тремя каналами BGR.
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    # Переводим с BGR в HSV

    hue = hsvC[0][0][0]  # Get the hue value
    # hsvC[0][0] Берём пиксель в позиции (0,0) — единственный пиксель в нашей «1×1 картинке».
    # Следующее значение [0] берём первый канал этого пикселя, то есть именно H (оттенок) Из HSV.[H]SV

    # Handle red hue wrap-around
    if hue >= 165:  # Upper limit for divided red hue
    # Проверяем, не слишком ли большой оттенок (≥ 165).
    # Оттенки около 0 и около 180 — это красный.
    # Числа ≥ 165 означают, что мы уже близки к концу круга и скоро снова на красном.
     
        
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        # Оттенок на 10 единиц ниже, чем наш hue.
        # Насыщенность и яркость минимум 100 (чтобы не захватить черно-белые/тусклые цвета).
        
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
       
        # Оттенок до 180 (максимум в OpenCV).
        # Насыщенность и яркость до 255 (максимум).
        # dtype=np.uint8 — снова указываем, что всё хранится в байтах 0–255.
        
        
    elif hue <= 15:  # Lower limit for divided red hue
    # elif — «иначе, если». Проверяет если условие if были не выполнены
    # Проверяем, не слишком ли маленький оттенок (≤ 15), то есть снова красный, но в начале круга.
        
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    # Аналогично предыдущему, но границы «сдвинуты» с начала:
    # lowerLimit = [0, 100, 100] — не ниже нуля.
    # upperLimit = [hue+10, 255, 255] — оттенок чуть правее нашего красного.
    
    else:
    # 🔹 else – это «иначе» else выполняется, если ни одно условие выше не сработало
    # 🔹 Выполняется, когда предыдущее условие или блок кода не сработал / завершился без ошибок / завершился полностью (в цикле).
        
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    # Если оттенок не в «красной зоне» (не ≤ 15 и не ≥ 165), то это какой-то другой цвет.
    # Просто берём диапазон ± 10 вокруг нашего оттенка:
    return lowerLimit, upperLimit
# Функция возвращает две переменные: lowerLimit и upperLimit.
# Представь: ты просишь у функции «дай мне границы по HSV для этого цвета», а она возвращает два массива.

In [3]:
my_color = [255, 0, 0]  # green in BGR colorspace
cap = cv2.VideoCapture(0)

In [4]:
while True:
    ret, frame = cap.read()

    hsvImage = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lowerLimit, upperLimit = get_limits(color=my_color)

    mask = cv2.inRange(hsvImage, lowerLimit, upperLimit)
    # cv2.inRange() создаёт черно-белую маску:
    #Белый цвет (255) – где HSV пиксели попадают в диапазон зелёного.
    #Чёрный цвет (0) – где пиксели вне диапазона.
    
    mask_ = Image.fromarray(mask)
    # Image.fromarray() превращает массив NumPy (маску) в объект PIL Image, чтобы можно было использовать метод getbbox().
    
    bbox = mask_.getbbox()
    # getbbox() возвращает координаты (x1, y1, x2, y2), 
        # которые образуют минимальный прямоугольник, охватывающий все белые (255) области маски.
    
    # Если нашего цвета в объекте нет, возвращает None.
    
    if bbox is not None:
    #Если bbox не равно None, значит, наш цвет в объекте найден.
        
        x1, y1, x2, y2 = bbox
        # Извлекаются координаты углов прямоугольника из переменной bbox
        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
        # Рисуем прямоугольник

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()

cv2.destroyAllWindows()